<a href="https://www.kaggle.com/code/somewhatjustin/titanic?scriptVersionId=164933266" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
'''
TODO: 
- make cabin category more useful
'''

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

In [ ]:
# Combine parch and sibsp into fam
train_data['fam'] = train_data['Parch']+train_data['SibSp']
test_data['fam'] = test_data['Parch']+test_data['SibSp']

def isAlone(fam):
    if fam == 0:
        return True
    else:
        return False

# Check if alone
train_data['alone'] = train_data['fam'].apply(isAlone)
test_data['alone'] = test_data['fam'].apply(isAlone)

In [ ]:
# Give Titles
def categorize_name(name):
    if 'Mr.' in name:
        return 'Mr'
    elif 'Master' in name:
        return 'Master'
    elif 'Col.' in name or 'Major.' in name or 'Capt.' in name:
        return 'Military'
    elif 'Miss' in name or 'Ms.' in name or 'Mlle.' in name:
        return 'Miss'
    elif 'Mrs.' in name or 'Mme.' in name:
        return 'Misses'
    elif 'Rev.' in name:
        return 'Reverend'
    elif 'Dr.' in name:
        return 'Doctor'
    elif 'Don.' in name or 'Lady.' in name or 'Sir.' in name or 'Countess.' in name or 'Jonkheer.' in name:
        return 'Royalty'
    else:
        return 'Other'

train_data['Title'] = train_data['Name'].apply(categorize_name)
test_data['Title'] = test_data['Name'].apply(categorize_name)

train_data[train_data['Title']=="Other"].head()

In [ ]:
# Bucket Age

# Fill NaNs without using inplace=True
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].mean())
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].mean())

# Create age buckets
age_buckets = np.linspace(0,80,20)
train_data['Age_bucket'] = pd.cut(train_data['Age'], age_buckets)
test_data['Age_bucket'] = pd.cut(test_data['Age'], age_buckets)

buckets, blah = pd.factorize(train_data['Age_bucket'])
train_data['Age_bucket'] = buckets
buckets, blah = pd.factorize(test_data['Age_bucket'])
test_data['Age_bucket'] = buckets

#train_data['Age_normalized'] = scaler.fit_transform(train_data[['Age']].values.reshape(-1, 1))
#test_data['Age_normalized'] = scaler.transform(test_data[['Age']].values.reshape(-1, 1))

train_data.head(30)

# Note: Use scaler.fit_transform() on the training data to fit the scaler and transform the data.
# Use scaler.transform() on the test data to apply the same scaling based on the training data.

In [ ]:
# Fare Normalization

# Fill NaNs without using inplace=True
train_data['Fare'] = train_data['Fare'].fillna(train_data['Fare'].median())
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())

In [ ]:
# Passenger Class Normalization

class_mapping = {1: "First", 2: "Second", 3: "Third"}
train_data['Pclass'] = train_data['Pclass'].map(class_mapping)
test_data['Pclass'] = test_data['Pclass'].map(class_mapping)

In [ ]:
# Make sex binary

# 2: female
# 1: male

sex_mapping = {"male": 1, "female": 2}
train_data['Sex'] = train_data['Sex'].map(sex_mapping)
test_data['Sex'] = test_data['Sex'].map(sex_mapping)

In [ ]:
# Cabin Classifier

def cabin_category(cabin):
    # Check for missing cabin information
    if pd.isnull(cabin):
        return 'Other'
    else:
        # Search for specific letters in the cabin string
        for letter in ['A', 'B', 'C', 'D', 'E', 'F']:
            if letter in cabin:
                return letter
        # Return 'Other' if none of the specific letters are found
        return 'Other'

# Apply the function to create a new 'Cabin_Category' column
train_data['Cabin_Category'] = train_data['Cabin'].apply(cabin_category)
test_data['Cabin_Category'] = test_data['Cabin'].apply(cabin_category)

In [ ]:
# Analysis

# Now create a pivot table with the new 'Cabin_Category'
pivot_table = pd.crosstab(index=train_data['Survived'], columns=train_data['Cabin_Category'])

# Display the pivot table
print(pivot_table)

In [ ]:
y = train_data["Survived"]

#features = ["Pclass", "Sex", "Age_bucket", "Fare", "alone", "Embarked", "fam", "Cabin_Category"]
features = ["Pclass", "Sex", "Age", "Fare", "alone", "Embarked", "fam", "Title"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])


# Ensure X_test has all the columns that X has, and fill missing ones with 0
for column in X.columns:
    if column not in X_test.columns:
        X_test[column] = 0  # Adding missing column in test data with default value of 0

# Align the order of X_test columns to match X
X_test = X_test[X.columns]

#model = GradientBoostingClassifier(n_estimators=150, max_depth=3).fit(X, y)
model = RandomForestClassifier(max_features=None, n_estimators=83).fit(X,y)
submissionPredictions = model.predict(X_test)

In [ ]:
# CHECK FOR OVERFITTING
from sklearn.metrics import accuracy_score

training_predictions = model.predict(X)
training_truth = y.to_numpy()

accuracy = accuracy_score(training_truth, training_predictions)
print(f"Accuracy:{accuracy}")

In [ ]:
# CROSS-VALIDATION
from sklearn.model_selection import GridSearchCV

n_estimators = np.linspace(10, 100, 50, dtype="int")
para_grid = {'n_estimators': n_estimators, 'max_features': ["sqrt", "log2", None, 1, 2, 3, 4, 5, 6, 7, 8]}

print(n_estimators)

#clf = GridSearchCV(estimator=model, param_grid=para_grid, cv=5, n_jobs=-1)
#clf.fit(X,y)

In [ ]:
'''
sorted_keys = sorted(clf.cv_results_.keys())  # Uncomment after fitting
print(sorted_keys)  # Uncomment after fitting

# Assuming `clf` is your GridSearchCV object and has been fitted with data
best_model = clf.best_estimator_

# To get the best score achieved during the cross-validation
best_score = clf.best_score_

# To see the parameters that yielded the best results
best_params = clf.best_params_

print("Best Score:", best_score)
print("Best Parameters:", best_params)
print("Best Model:", best_model)
'''

In [ ]:
import matplotlib.pyplot as plt

# Accessing feature importances
feature_importances = model.feature_importances_

# Visualizing feature importances
features = X.columns
indices = np.argsort(feature_importances)[::-1]

plt.figure(figsize=(12, 6))
plt.title("Feature Importances in Gradient Boosting Classifier")
plt.bar(range(X.shape[1]), feature_importances[indices], align="center")
plt.xticks(range(X.shape[1]), features[indices], rotation=90)
plt.xlabel("Feature")
plt.ylabel("Importance")
plt.show()

In [ ]:
# CREATE PREDICTIONS
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': submissionPredictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")